# oliver model 
## random and grid search parameter tuning


1. random search for RANDOM_MAX_EVALS times
2. grid search around random search's best parameter

In [1]:
RANDOM_MAX_EVALS = 50
GRID_MAX_EVALS = 50
N_FOLDS = 5

In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from sklearn.model_selection import KFold, train_test_split
from lightgbm import LGBMClassifier

import time
import gc
from tqdm import tqdm

import random
import lightgbm as lgb

In [3]:
!ls ../input

HomeCredit_columns_description.csv  bureau_balance.csv
POS_CASH_balance.csv		    credit_card_balance.csv
application_test.csv		    installments_payments.csv
application_train.csv		    previous_application.csv
bureau.csv			    sample_submission.csv


In [4]:
pd.options.display.max_columns = None

In [5]:
def build_model_input():
    buro_bal = pd.read_csv('../input/bureau_balance.csv')
    print('Buro bal shape : ', buro_bal.shape)
    
    # 1hot encoding
    print('transform to dummies')
    buro_bal = pd.concat([buro_bal, pd.get_dummies(buro_bal.STATUS, prefix='buro_bal_status')], axis=1).drop('STATUS', axis=1)
    
    # 'SK_ID_BUREAU', 'MONTHS_BALANCE'をカウント
    print('Counting buros')
    buro_counts = buro_bal[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()
    buro_bal['buro_count'] = buro_bal['SK_ID_BUREAU'].map(buro_counts['MONTHS_BALANCE'])
    
    # balance をaverage
    print('averaging buro bal')
    avg_buro_bal = buro_bal.groupby('SK_ID_BUREAU').mean()
    
    avg_buro_bal.columns = ['avg_buro_' + f_ for f_ in avg_buro_bal.columns]
    del buro_bal
    gc.collect()
    
    print('Read Bureau')
    buro = pd.read_csv('../input/bureau.csv')
    
    # bureau を 1hot encoding
    print('Go to dummies')
    buro_credit_active_dum = pd.get_dummies(buro.CREDIT_ACTIVE, prefix='ca_')
    buro_credit_currency_dum = pd.get_dummies(buro.CREDIT_CURRENCY, prefix='cu_')
    buro_credit_type_dum = pd.get_dummies(buro.CREDIT_TYPE, prefix='ty_')
    
    # 1hotをマージ
    buro_full = pd.concat([buro, buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum], axis=1)
    # buro_full.columns = ['buro_' + f_ for f_ in buro_full.columns]
    
    del buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum
    gc.collect()
    
    print('Merge with buro avg')
    buro_full = buro_full.merge(right=avg_buro_bal.reset_index(), how='left', on='SK_ID_BUREAU', suffixes=('', '_bur_bal'))
    
    # breauの数のカウント
    print('Counting buro per SK_ID_CURR')
    nb_bureau_per_curr = buro_full[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
    buro_full['SK_ID_BUREAU'] = buro_full['SK_ID_CURR'].map(nb_bureau_per_curr['SK_ID_BUREAU'])
    
    print('Averaging bureau')
    avg_buro = buro_full.groupby('SK_ID_CURR').mean()
    print(avg_buro.head())
    
    del buro, buro_full
    gc.collect()
    
    print('Read prev')
    prev = pd.read_csv('../input/previous_application.csv')
    
    prev_cat_features = [
        f_ for f_ in prev.columns if prev[f_].dtype == 'object'
    ]
    
    print('Go to dummies')
    prev_dum = pd.DataFrame()
    for f_ in prev_cat_features:
        prev_dum = pd.concat([prev_dum, pd.get_dummies(prev[f_], prefix=f_).astype(np.uint8)], axis=1)
    
    prev = pd.concat([prev, prev_dum], axis=1)
    
    del prev_dum
    gc.collect()
    
    print('Counting number of Prevs')
    nb_prev_per_curr = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
    prev['SK_ID_PREV'] = prev['SK_ID_CURR'].map(nb_prev_per_curr['SK_ID_PREV'])
    
    print('Averaging prev')
    avg_prev = prev.groupby('SK_ID_CURR').mean()
    print(avg_prev.head())
    del prev
    gc.collect()
    
    print('Reading POS_CASH')
    pos = pd.read_csv('../input/POS_CASH_balance.csv')
    
    print('Go to dummies')
    pos = pd.concat([pos, pd.get_dummies(pos['NAME_CONTRACT_STATUS'])], axis=1)
    
    print('Compute nb of prevs per curr')
    nb_prevs = pos[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
    pos['SK_ID_PREV'] = pos['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
    
    print('Go to averages')
    avg_pos = pos.groupby('SK_ID_CURR').mean()
    
    del pos, nb_prevs
    gc.collect()
    
    print('Reading CC balance')
    cc_bal = pd.read_csv('../input/credit_card_balance.csv')
    
    print('Go to dummies')
    cc_bal = pd.concat([cc_bal, pd.get_dummies(cc_bal['NAME_CONTRACT_STATUS'], prefix='cc_bal_status_')], axis=1)
    
    nb_prevs = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
    cc_bal['SK_ID_PREV'] = cc_bal['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
    
    print('Compute average')
    avg_cc_bal = cc_bal.groupby('SK_ID_CURR').mean()
    avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]
    
    del cc_bal, nb_prevs
    gc.collect()
    
    print('Reading Installments')
    inst = pd.read_csv('../input/installments_payments.csv')
    nb_prevs = inst[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
    inst['SK_ID_PREV'] = inst['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
    
    avg_inst = inst.groupby('SK_ID_CURR').mean()
    avg_inst.columns = ['inst_' + f_ for f_ in avg_inst.columns]
    
    print('Read data and test')
    data = pd.read_csv('../input/application_train.csv')
    test = pd.read_csv('../input/application_test.csv')
    print('Shapes : ', data.shape, test.shape)
    
    y = data['TARGET']
    del data['TARGET']
    
    categorical_feats = [
        f for f in data.columns if data[f].dtype == 'object'
    ]
    categorical_feats
    for f_ in categorical_feats:
        data[f_], indexer = pd.factorize(data[f_])
        test[f_] = indexer.get_indexer(test[f_])
        
    data = data.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')
    test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')
    
    data = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
    test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
    
    data = data.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
    test = test.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
    
    data = data.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
    test = test.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
    
    data = data.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')
    test = test.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')
    
    del avg_buro, avg_prev
    gc.collect()

    return data, test, y

In [6]:
gc.enable()
# Build model inputs
data, test, y = build_model_input()

# Split into training and testing data
train_features, test_features, train_labels, test_labels = train_test_split(data, y, test_size = 6000, random_state = 50)
# Create a training and testing dataset
train_set = lgb.Dataset(data = train_features, label = train_labels)
test_set = lgb.Dataset(data = test_features, label = test_labels)

Buro bal shape :  (27299925, 3)
transform to dummies
Counting buros
averaging buro bal
Read Bureau
Go to dummies
Merge with buro avg
Counting buro per SK_ID_CURR
Averaging bureau
            SK_ID_BUREAU  DAYS_CREDIT  CREDIT_DAY_OVERDUE  \
SK_ID_CURR                                                  
100001               7.0  -735.000000                 0.0   
100002               8.0  -874.000000                 0.0   
100003               4.0 -1400.750000                 0.0   
100004               2.0  -867.000000                 0.0   
100005               3.0  -190.666667                 0.0   

            DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  \
SK_ID_CURR                                                                   
100001                82.428571        -825.500000                     NaN   
100002              -349.000000        -697.500000                1681.029   
100003              -544.500000       -1097.333333                   0.000   
100

Go to dummies
Counting number of Prevs
Averaging prev
            SK_ID_PREV  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  \
SK_ID_CURR                                                         
100001             1.0     3951.000         24835.50    23787.00   
100002             1.0     9251.775        179055.00   179055.00   
100003             3.0    56553.990        435436.50   484191.00   
100004             1.0     5357.250         24282.00    20106.00   
100005             2.0     4813.200         22308.75    20076.75   

            AMT_DOWN_PAYMENT  AMT_GOODS_PRICE  HOUR_APPR_PROCESS_START  \
SK_ID_CURR                                                               
100001                2520.0          24835.5                13.000000   
100002                   0.0         179055.0                 9.000000   
100003                3442.5         435436.5                14.666667   
100004                4860.0          24282.0                 5.000000   
100005                446

Go to dummies
Compute nb of prevs per curr
Go to averages
Reading CC balance
Go to dummies
Compute average
Reading Installments
Read data and test
Shapes :  (307511, 122) (48744, 121)


In [7]:
def objective(hyperparameters, iteration):
    """Objective function for grid and random search. Returns
       the cross validation score from a set of hyperparameters."""
    
    # Number of estimators will be found using early stopping
    if 'n_estimators' in hyperparameters.keys():
        del hyperparameters['n_estimators']
    
     # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, train_set, num_boost_round = 10000, nfold = N_FOLDS, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 42)
    
    # results to retun
    score = cv_results['auc-mean'][-1]
    estimators = len(cv_results['auc-mean'])
    hyperparameters['n_estimators'] = estimators 
    
    return [score, hyperparameters, iteration]

In [8]:
def random_search(param_rand, max_evals = RANDOM_MAX_EVALS):
    """Random search for hyperparameter optimization"""
    
    # Dataframe for results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                                  index = list(range(RANDOM_MAX_EVALS)))
    
    # Keep searching until reach max evaluations
    for i in tqdm(range(RANDOM_MAX_EVALS)):
        
        # Choose random hyperparameters
        hyperparameters = {k: random.sample(v, 1)[0] for k, v in param_rand.items()}
        hyperparameters['subsample'] = 1.0 if hyperparameters['boosting_type'] == 'goss' else hyperparameters['subsample']

        # Evaluate randomly selected hyperparameters
        eval_results = objective(hyperparameters, i)
        
        results.loc[i, :] = eval_results
    
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)
    return results 

In [9]:
# Hyperparameter grid
param_rand = {
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': list(range(20, 150)),
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 1000)),
    'subsample_for_bin': list(range(20000, 300000, 20000)),
    'min_child_samples': list(range(20, 500, 5)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10)),
    'subsample': list(np.linspace(0.5, 1, 100)),
    'is_unbalance': [True, False]
}

In [10]:
ra_st = time.time()
random_results = random_search(param_rand)
print("%f s" % (time.time() - ra_st))

100%|██████████| 50/50 [15:02:54<00:00, 1083.50s/it]  

54175.000683 s


In [11]:
random_results

,index,score,params,iteration
0,13,0.781172,"{'boosting_type': 'gbdt', 'num_leaves': 137, '...",13
1,26,0.780928,"{'boosting_type': 'gbdt', 'num_leaves': 54, 'l...",26
2,44,0.780838,"{'boosting_type': 'gbdt', 'num_leaves': 21, 'l...",44
3,6,0.780579,"{'boosting_type': 'gbdt', 'num_leaves': 122, '...",6
4,28,0.779734,"{'boosting_type': 'dart', 'num_leaves': 31, 'l...",28
5,49,0.779719,"{'boosting_type': 'gbdt', 'num_leaves': 31, 'l...",49
6,33,0.779691,"{'boosting_type': 'gbdt', 'num_leaves': 148, '...",33
7,7,0.779655,"{'boosting_type': 'dart', 'num_leaves': 51, 'l...",7
8,10,0.779552,"{'boosting_type': 'dart', 'num_leaves': 88, 'l...",10
9,40,0.779494,"{'boosting_type': 'dart', 'num_leaves': 89, 'l...",40


In [12]:
param_rand_best = random_results['params'][0]

In [13]:
param_rand_best

{'boosting_type': 'gbdt',
 'num_leaves': 137,
 'learning_rate': 0.005715155645572393,
 'subsample_for_bin': 100000,
 'min_child_samples': 465,
 'reg_alpha': 0.42857142857142855,
 'reg_lambda': 0.5306122448979591,
 'colsample_bytree': 0.9555555555555555,
 'subsample': 0.8787878787878789,
 'is_unbalance': True,
 'metric': 'auc',
 'verbose': 1,
 'n_estimators': 2050}

In [14]:
param_grid = {
    'boosting_type': [param_rand_best['boosting_type']],
    'num_leaves': list(range(max(1, param_rand_best['num_leaves'] - 10), param_rand_best['num_leaves'] +11, 10)),
    'learning_rate': list(np.logspace(np.log10(param_rand_best['learning_rate'] * 0.9 ), np.log10(param_rand_best['learning_rate'] * 1.1), base = 10, num = 3)),
    'subsample_for_bin': list(range(max(20000, param_rand_best['subsample_for_bin'] - 20000), param_rand_best['subsample_for_bin'] + 20001, 20000)),
    'min_child_samples': list(range(param_rand_best['min_child_samples'], 500, 3)),
    'reg_alpha': list(np.linspace(param_rand_best['reg_alpha'] * 0.9, min(param_rand_best['reg_alpha'] * 1.1, 1.0), 3)),
    'reg_lambda': list(np.linspace(param_rand_best['reg_lambda'] * 0.9, min(param_rand_best['reg_lambda'] * 1.1, 1.0), 3)),
    'colsample_bytree': list(np.linspace(param_rand_best['colsample_bytree'] * 0.9, min(param_rand_best['colsample_bytree'] * 1.1, 1.0), 3)),
    'subsample': list(np.linspace(param_rand_best['subsample'] * 0.9, min(param_rand_best['subsample'] * 0.9, 1.0), 3)),
    'is_unbalance': [param_rand_best['is_unbalance']]
}

In [15]:
import itertools

def grid_search(param_grid, max_evals = GRID_MAX_EVALS):
    """Grid search algorithm (with limit on max evals)"""
    
    # Dataframe to store results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(GRID_MAX_EVALS)))
    
    # https://codereview.stackexchange.com/questions/171173/list-all-possible-permutations-from-a-python-dictionary-of-lists
    keys, values = zip(*param_grid.items())
    
    i = 0
    
    # Iterate through every possible combination of hyperparameters
    for v in tqdm(itertools.product(*values)):
        
        # Create a hyperparameter dictionary
        hyperparameters = dict(zip(keys, v))
        
        # Set the subsample ratio accounting for boosting type
        hyperparameters['subsample'] = 1.0 if hyperparameters['boosting_type'] == 'goss' else hyperparameters['subsample']
        
        # Evalute the hyperparameters
        eval_results = objective(hyperparameters, i)
        
        results.loc[i, :] = eval_results
        
        i += 1
        
        # Normally would not limit iterations
        if i > GRID_MAX_EVALS:
            break
       
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)
    
    return results

In [16]:
gr_st = time.time()
grid_results = grid_search(param_grid)
print("grid search %f s" % (time.time() - gr_st))
print('The best validation score was {:.5f}'.format(grid_results.loc[0, 'score']))
print('\nThe best hyperparameters were:')

import pprint
pprint.pprint(grid_results.loc[0, 'params'])

50it [37:17:29, 2685.00s/it]

grid search 136964.109816 s
The best validation score was 0.78160

The best hyperparameters were:
{'boosting_type': 'gbdt',
 'colsample_bytree': 0.86,
 'is_unbalance': True,
 'learning_rate': 0.005143640081015152,
 'metric': 'auc',
 'min_child_samples': 465,
 'n_estimators': 2329,
 'num_leaves': 127,
 'reg_alpha': 0.3857142857142857,
 'reg_lambda': 0.4775510204081632,
 'subsample': 0.790909090909091,
 'subsample_for_bin': 80000,
 'verbose': 1}
